In [1]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

# Co potrzebujemy do opisu pozycji na ETF?

Spróbujmy tak:
1. Czas otwarcia pozycji
2. Ticker instrumentu

Dodatkowe parametry:
1. Spread (walutowy i na instrumentach), hardcoded
2. Marża za przewalutowanie (`0.5%`), MARGIN

Przykład: otwieramy pozycję na `4GLD.DE`

In [81]:
def get_exchange_rate(currency: str, time: pd.Timestamp, type: str | None, spread: float | None, margin: float | None):
    
    # Assertions
    assert type in ('bid', 'ask', None), ''
    if type in ('bid', 'ask'): assert spread is not None and margin is not None, ''
    
    # Code
    if currency not in ('PLN'): # jeżeli kupujemy instrument w obcej walucie
    
        currency_ticker = f'{currency}PLN=X'
        currency_ticker = yf.Ticker(currency_ticker)
        
        currency_history = currency_ticker.history(
            start=time.floor('h'), 
            end=time.ceil('h'),
            interval='1m'
        )
        
        try:
            currency_history = currency_history[currency_history.index == time].iloc[0]
        except IndexError as e:
            print(str(e).strip())
            currency_history = currency_history.iloc[-1]
        
        if type == 'ask':
            exchange_rate = round((currency_history['High'] + spread / 2) * (1 + margin), 4)
        elif type == 'bid':
            exchange_rate = round((currency_history['Low'] - spread / 2) * (1 - margin), 4)
        else:
            exchange_rate = 0.5*(currency_history['High'] + currency_history['Low'])
        
    else:
        exchange_rate = None
        
    return exchange_rate


get_exchange_rate(currency='EUR', time=pd.Timestamp('2026-02-02T09:22', tz='Europe/Warsaw'), type='ask', spread=0.002, margin=0.005)

np.float64(4.2321)

In [82]:
def get_ticker_price(ticker: str, time: pd.Timestamp, type: str, spread: float):
    
    # Assertions
    assert type in ('bid', 'ask', None), ''
    if type in ('bid', 'ask'): assert spread is not None, ''
    
    # Code
    
    ticker = yf.Ticker(ticker)
    
    ticker_history = ticker.history(
        start=time.floor('h'), 
        end=time.ceil('h'),
        interval='1m'
    )
    
    try:
        ticker_history = ticker_history[ticker_history.index == time].iloc[0]
    except IndexError as e:
        print(str(e).strip())
        ticker_history = ticker_history.iloc[-1]
        
    if type == 'ask':
        ticker_price = 0.5*(ticker_history['High'] + ticker_history['Low']) * (1 + spread)
    elif type == 'bid':
        ticker_price = 0.5*(ticker_history['High'] + ticker_history['Low']) * (1 - spread)
    else:
        ticker_price = 0.5*(ticker_history['High'] + ticker_history['Low'])

    return ticker_price

get_ticker_price(ticker='4GLD.DE', time=pd.Timestamp('2026-02-02T09:22', tz='Europe/Warsaw'), type='ask', spread=0.001)

np.float64(125.87074393081663)

In [101]:
position_info = pd.read_csv('data/positions.csv').iloc[1]
position_info

ticker_xtb                                        IUSQ.DE
ticker_yf                                         IUSQ.DE
opening_time                    2026-02-02 09:04:00+01:00
volume                                             0.6275
action                                      position_open
opening_price                                       93.27
opening_exchange_rate                               4.235
last_modified            2026-02-05 17:39:12.553721+01:00
Name: 1, dtype: object

In [102]:
# Input
position_opening_time = position_info['opening_time']
volume = position_info['volume']
ticker = position_info['ticker_yf']

MARGIN = 0.005

# Code
position_opening_time = pd.Timestamp(position_opening_time, tz='Europe/Warsaw')

info = yf.Ticker(ticker).info

currency = info['currency']
timezone = info['exchangeTimezoneName']
position_opening_time = position_opening_time.tz_convert(timezone)

# Kurs walutowy przy otwarciu pozycji - wariant z pobraniem
position_opening_exchange_rate = get_exchange_rate(
    currency=currency, 
    time=position_opening_time, 
    type='ask', 
    spread=0.002, # hardcoded
    margin=MARGIN
)

if currency not in ('PLN'): print(f'{position_opening_exchange_rate/position_info['opening_exchange_rate'] - 1:.2%}')

# Kurs instrumentu przy otwarciu pozycji - wariant z pobraniem
position_opening_ticker_price = get_ticker_price(
    ticker=ticker, 
    time=pd.Timestamp('2026-02-02T09:22', tz='Europe/Warsaw'), 
    type='ask', 
    spread=0.002 # hardcoded
)

print(f'{position_opening_ticker_price/position_info['opening_price'] - 1:.2%}')

# Wartość pozycji przy otwarciu w PLN
if currency not in ('PLN'): position_opening_value = position_opening_ticker_price * position_opening_exchange_rate * volume
else: position_opening_value = position_opening_ticker_price * volume

position_opening_value

0.00%
0.29%


np.float64(248.59532477649964)

In [76]:
get_exchange_rate(
    currency='EUR', 
    time=pd.Timestamp.now(tz='Europe/Warsaw').floor(freq='min'), 
    type='ask', 
    spread=0.002, 
    margin=MARGIN
)

np.float64(4.2412)

In [98]:
get_ticker_price(
    ticker='ETFBTBSP.WA', 
    time=pd.Timestamp.now(tz='Europe/Warsaw').floor(freq='min'), 
    type=None,
    spread=None
)

single positional indexer is out-of-bounds


np.float64(229.5500030517578)